## Einführung

Diese Lektion behandelt:  
- Was Funktionsaufrufe sind und wofür sie verwendet werden  
- Wie man einen Funktionsaufruf mit OpenAI erstellt  
- Wie man einen Funktionsaufruf in eine Anwendung integriert  

## Lernziele

Nach Abschluss dieser Lektion wissen Sie, wie man Folgendes macht und verstehen:  

- Den Zweck der Verwendung von Funktionsaufrufen  
- Einrichtung von Funktionsaufrufen mit dem OpenAI-Dienst  
- Gestaltung effektiver Funktionsaufrufe für den Anwendungsfall Ihrer Anwendung


## Verständnis von Funktionsaufrufen

Für diese Lektion möchten wir eine Funktion für unser Bildungs-Startup entwickeln, die es Nutzern ermöglicht, einen Chatbot zu verwenden, um technische Kurse zu finden. Wir werden Kurse empfehlen, die ihrem Fähigkeitsniveau, ihrer aktuellen Rolle und der interessierenden Technologie entsprechen.

Um dies zu erreichen, verwenden wir eine Kombination aus:
 - `OpenAI`, um eine Chat-Erfahrung für den Nutzer zu schaffen
 - `Microsoft Learn Catalog API`, um Nutzern zu helfen, Kurse basierend auf der Anfrage des Nutzers zu finden
 - `Function Calling`, um die Anfrage des Nutzers zu erfassen und an eine Funktion zu senden, die die API-Anfrage durchführt.

Um zu beginnen, schauen wir uns an, warum wir überhaupt Function Calling verwenden möchten:

print("Nachrichten in der nächsten Anfrage:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # eine neue Antwort von GPT erhalten, bei der es die Funktionsantwort sehen kann


print(second_response.choices[0].message)


### Warum Funktionsaufrufe

Wenn Sie bereits eine andere Lektion in diesem Kurs abgeschlossen haben, verstehen Sie wahrscheinlich die Leistungsfähigkeit der Verwendung von Large Language Models (LLMs). Hoffentlich können Sie auch einige ihrer Einschränkungen erkennen.

Funktionsaufrufe sind eine Funktion des OpenAI-Dienstes, die entwickelt wurde, um die folgenden Herausforderungen zu bewältigen:

Inkonsistente Antwortformatierung:
- Vor Funktionsaufrufen waren die Antworten eines großen Sprachmodells unstrukturiert und inkonsistent. Entwickler mussten komplexen Validierungscode schreiben, um jede Variation der Ausgabe zu handhaben.

Begrenzte Integration mit externen Daten:
- Vor dieser Funktion war es schwierig, Daten aus anderen Teilen einer Anwendung in einen Chat-Kontext einzubinden.

Durch die Standardisierung von Antwortformaten und die nahtlose Integration mit externen Daten vereinfacht Funktionsaufrufe die Entwicklung und reduziert den Bedarf an zusätzlicher Validierungslogik.

Benutzer konnten keine Antworten wie „Wie ist das aktuelle Wetter in Stockholm?“ erhalten. Dies liegt daran, dass Modelle auf die Zeit beschränkt waren, zu der die Daten trainiert wurden.

Schauen wir uns das folgende Beispiel an, das dieses Problem veranschaulicht:

Angenommen, wir möchten eine Datenbank mit Studentendaten erstellen, damit wir ihnen den richtigen Kurs vorschlagen können. Unten haben wir zwei Beschreibungen von Studenten, die in den enthaltenen Daten sehr ähnlich sind.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Wir möchten dies an ein LLM senden, um die Daten zu analysieren. Dies kann später in unserer Anwendung verwendet werden, um es an eine API zu senden oder in einer Datenbank zu speichern.

Lassen Sie uns zwei identische Eingabeaufforderungen erstellen, in denen wir das LLM anweisen, welche Informationen für uns von Interesse sind:


Wir möchten dies an ein LLM senden, um die für unser Produkt wichtigen Teile zu analysieren. So können wir zwei identische Eingabeaufforderungen erstellen, um das LLM zu instruieren:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Nachdem wir diese beiden Eingabeaufforderungen erstellt haben, senden wir sie mit `openai.ChatCompletion` an das LLM. Wir speichern die Eingabeaufforderung in der Variablen `messages` und weisen die Rolle `user` zu. Dies soll eine Nachricht von einem Benutzer simulieren, die an einen Chatbot geschrieben wird.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Jetzt können wir beide Anfragen an das LLM senden und die erhaltene Antwort untersuchen.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Obwohl die Eingabeaufforderungen gleich sind und die Beschreibungen ähnlich sind, können wir unterschiedliche Formate der Eigenschaft `Grades` erhalten.

Wenn Sie die obige Zelle mehrmals ausführen, kann das Format `3.7` oder `3.7 GPA` sein.

Das liegt daran, dass das LLM unstrukturierte Daten in Form der geschriebenen Eingabeaufforderung aufnimmt und ebenfalls unstrukturierte Daten zurückgibt. Wir benötigen jedoch ein strukturiertes Format, damit wir wissen, was zu erwarten ist, wenn wir diese Daten speichern oder verwenden.

Durch die Verwendung von Funktionsaufrufen können wir sicherstellen, dass wir strukturierte Daten zurückerhalten. Beim Einsatz von Funktionsaufrufen ruft das LLM tatsächlich keine Funktionen auf oder führt sie aus. Stattdessen erstellen wir eine Struktur, der das LLM für seine Antworten folgen soll. Diese strukturierten Antworten verwenden wir dann, um zu wissen, welche Funktion in unseren Anwendungen ausgeführt werden soll.


![Funktionsaufruf-Flussdiagramm](../../../../translated_images/Function-Flow.083875364af4f4bb.de.png)


Wir können dann das, was von der Funktion zurückgegeben wird, nehmen und an das LLM zurücksenden. Das LLM wird dann in natürlicher Sprache antworten, um die Anfrage des Benutzers zu beantworten.


### Anwendungsfälle für die Verwendung von Funktionsaufrufen

**Externe Werkzeuge aufrufen**  
Chatbots sind hervorragend darin, Antworten auf Fragen von Nutzern zu geben. Durch die Verwendung von Funktionsaufrufen können die Chatbots Nachrichten von Nutzern nutzen, um bestimmte Aufgaben zu erledigen. Zum Beispiel kann ein Student den Chatbot bitten: „Sende eine E-Mail an meinen Dozenten, in der steht, dass ich mehr Unterstützung bei diesem Thema benötige“. Dies kann einen Funktionsaufruf an `send_email(to: string, body: string)` auslösen.

**API- oder Datenbankabfragen erstellen**  
Nutzer können Informationen in natürlicher Sprache finden, die in eine formatierte Abfrage oder API-Anfrage umgewandelt wird. Ein Beispiel hierfür könnte ein Lehrer sein, der fragt: „Wer sind die Studenten, die die letzte Aufgabe abgeschlossen haben“, was eine Funktion namens `get_completed(student_name: string, assignment: int, current_status: string)` aufrufen könnte.

**Strukturierte Daten erstellen**  
Nutzer können einen Textblock oder eine CSV-Datei nehmen und das LLM verwenden, um wichtige Informationen daraus zu extrahieren. Zum Beispiel kann ein Student einen Wikipedia-Artikel über Friedensabkommen in KI-Lernkarten umwandeln. Dies kann durch die Verwendung einer Funktion namens `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` erfolgen.


## 2. Erstellen Ihres ersten Funktionsaufrufs

Der Prozess zur Erstellung eines Funktionsaufrufs umfasst 3 Hauptschritte:  
1. Aufrufen der Chat Completions API mit einer Liste Ihrer Funktionen und einer Benutzernachricht  
2. Lesen der Antwort des Modells, um eine Aktion auszuführen, z. B. eine Funktion oder einen API-Aufruf auszuführen  
3. Einen weiteren Aufruf an die Chat Completions API mit der Antwort Ihrer Funktion machen, um diese Informationen zu verwenden, um eine Antwort für den Benutzer zu erstellen.


![Ablauf eines Funktionsaufrufs](../../../../translated_images/LLM-Flow.3285ed8caf4796d7.de.png)


### Elemente eines Funktionsaufrufs

#### Benutzereingabe

Der erste Schritt besteht darin, eine Benutzernachricht zu erstellen. Diese kann dynamisch zugewiesen werden, indem der Wert einer Texteingabe übernommen wird, oder Sie können hier einen Wert zuweisen. Wenn Sie zum ersten Mal mit der Chat Completions API arbeiten, müssen wir die `role` und den `content` der Nachricht definieren.

Die `role` kann entweder `system` (Regeln erstellen), `assistant` (das Modell) oder `user` (der Endbenutzer) sein. Für den Funktionsaufruf weisen wir dies als `user` und eine Beispiel-Frage zu.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funktionen erstellen.

Als Nächstes definieren wir eine Funktion und die Parameter dieser Funktion. Wir verwenden hier nur eine Funktion namens `search_courses`, aber Sie können mehrere Funktionen erstellen.

**Wichtig**: Funktionen sind in der Systemnachricht an das LLM enthalten und werden auf die Anzahl der verfügbaren Tokens angerechnet.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitionen** 

Die Funktionsdefinitionsstruktur hat mehrere Ebenen, jede mit eigenen Eigenschaften. Hier ist eine Aufschlüsselung der verschachtelten Struktur:

**Eigenschaften der obersten Funktionsebene:**

`name` - Der Name der Funktion, die aufgerufen werden soll. 

`description` - Dies ist die Beschreibung, wie die Funktion funktioniert. Hier ist es wichtig, spezifisch und klar zu sein. 

`parameters` - Eine Liste von Werten und Formaten, die das Modell in seiner Antwort erzeugen soll. 

**Eigenschaften des Parameterobjekts:**

`type` - Der Datentyp des Parameterobjekts (normalerweise "object")

`properties` - Liste der spezifischen Werte, die das Modell für seine Antwort verwenden wird. 

**Eigenschaften einzelner Parameter:**

`name` - Implizit durch den Eigenschaftsschlüssel definiert (z. B. "role", "product", "level")

`type` - Der Datentyp dieses spezifischen Parameters (z. B. "string", "number", "boolean") 

`description` - Beschreibung des spezifischen Parameters 

**Optionale Eigenschaften:**

`required` - Ein Array, das auflistet, welche Parameter für den Abschluss des Funktionsaufrufs erforderlich sind.


### Den Funktionsaufruf machen  
Nachdem wir eine Funktion definiert haben, müssen wir sie nun im Aufruf der Chat Completion API einbinden. Dies tun wir, indem wir `functions` zur Anfrage hinzufügen. In diesem Fall `functions=functions`.  

Es gibt auch die Möglichkeit, `function_call` auf `auto` zu setzen. Das bedeutet, dass wir das LLM entscheiden lassen, welche Funktion basierend auf der Benutzernachricht aufgerufen werden soll, anstatt dies selbst zuzuweisen.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Schauen wir uns nun die Antwort an und sehen, wie sie formatiert ist:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Sie können sehen, dass der Name der Funktion aufgerufen wird und dass das LLM anhand der Benutzernachricht die Daten gefunden hat, um die Argumente der Funktion zu füllen.


## 3. Integration von Funktionsaufrufen in eine Anwendung. 


Nachdem wir die formatierte Antwort des LLM getestet haben, können wir diese nun in eine Anwendung integrieren. 

### Steuerung des Ablaufs 

Um dies in unsere Anwendung zu integrieren, gehen wir wie folgt vor: 

Zuerst rufen wir die OpenAI-Dienste auf und speichern die Nachricht in einer Variablen namens `response_message`. 


In [ ]:
response_message = response.choices[0].message

Jetzt definieren wir die Funktion, die die Microsoft Learn API aufruft, um eine Liste von Kursen zu erhalten:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Als bewährte Methode werden wir dann prüfen, ob das Modell eine Funktion aufrufen möchte. Danach erstellen wir eine der verfügbaren Funktionen und ordnen sie der Funktion zu, die aufgerufen wird.  
Anschließend nehmen wir die Argumente der Funktion und ordnen sie den Argumenten des LLM zu.

Zuletzt fügen wir die Funktionsaufrufnachricht und die Werte, die durch die `search_courses`-Nachricht zurückgegeben wurden, an. Dies gibt dem LLM alle Informationen, die es benötigt, um in natürlicher Sprache auf den Benutzer zu antworten.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Jetzt senden wir die aktualisierte Nachricht an das LLM, damit wir eine Antwort in natürlicher Sprache anstelle einer API JSON-formatierten Antwort erhalten können.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code-Herausforderung

Großartige Arbeit! Um dein Lernen über OpenAI Function Calling fortzusetzen, kannst du Folgendes erstellen: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - Weitere Parameter der Funktion, die Lernenden helfen könnten, mehr Kurse zu finden. Die verfügbaren API-Parameter findest du hier:  
 - Erstelle einen weiteren Funktionsaufruf, der mehr Informationen vom Lernenden wie seine Muttersprache entgegennimmt  
 - Erstelle eine Fehlerbehandlung, wenn der Funktionsaufruf und/oder der API-Aufruf keine geeigneten Kurse zurückgibt


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ursprungssprache ist als maßgebliche Quelle zu betrachten. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
